## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [4]:
import pandas as pd

In [5]:
df=pd.read_csv(r'train.csv')

In [6]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
###Drop Nan Values
df=df.dropna()


In [8]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [9]:
## Get the Dependent features
y=df['label']

In [10]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [11]:
X.shape

(18285, 4)

In [12]:
y.shape

(18285,)

In [13]:
import tensorflow as tf

In [14]:
tf.__version__

'2.15.0'

In [15]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [16]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
messages.reset_index(inplace=True)

In [20]:
import nltk
import re
from nltk.corpus import stopwords

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [22]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[361, 4673, 4796, 502, 3392, 2434, 2029, 1050, 1031, 4638],
 [2020, 1167, 432, 416, 4023, 946, 2251],
 [1405, 1325, 3312, 3645],
 [594, 158, 3125, 1260, 1297, 3698],
 [4613, 4023, 3724, 3939, 1826, 4469, 4023, 4294, 4593, 2073],
 [1814,
  1604,
  1289,
  3408,
  4567,
  262,
  2420,
  3273,
  4663,
  1666,
  4280,
  4908,
  682,
  2848,
  2251],
 [2303, 4848, 95, 3644, 4840, 135, 1079, 868, 1780, 275, 3778],
 [1888, 1967, 680, 3862, 2476, 921, 262, 4242, 1780, 275, 3778],
 [2734, 4271, 2421, 4907, 956, 4032, 1858, 2572, 262, 47],
 [1619, 4738, 1267, 259, 2796, 838, 4785, 2018],
 [260, 614, 2807, 1130, 2106, 1992, 4276, 790, 4854, 1845, 744],
 [1260, 4223, 3392, 4032, 262, 2476],
 [4267, 4697, 592, 1513, 1181, 3161, 713, 4365, 2195],
 [3822, 433, 1207, 1432, 388, 2404, 2180, 1780, 275, 3778],
 [1699, 3445, 2983, 3751, 1116, 1780, 275, 3778],
 [2322, 2983, 1664, 1280, 3433, 3092, 1555, 1273, 3167, 1112],
 [1017, 933, 1167],
 [2817, 1469, 1877, 3868, 262, 2694, 2562, 2251],
 [210, 2988, 

### Embedding Representation

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1050 1031 4638]
 [   0    0    0 ... 4023  946 2251]
 [   0    0    0 ... 1325 3312 3645]
 ...
 [   0    0    0 ... 1780  275 3778]
 [   0    0    0 ... 3392 3006  232]
 [   0    0    0 ... 2390 1841  541]]


In [26]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  361,
       4673, 4796,  502, 3392, 2434, 2029, 1050, 1031, 4638], dtype=int32)

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [28]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [29]:
len(embedded_docs),y.shape

(18285, (18285,))

In [30]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [31]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [33]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 22s 85ms/step - loss: 0.3006 - accuracy: 0.8597 - val_loss: 0.2058 - val_accuracy: 0.9203
Epoch 2/10
192/192 [==============================] - 16s 85ms/step - loss: 0.1361 - accuracy: 0.9484 - val_loss: 0.2031 - val_accuracy: 0.9193
Epoch 3/10
192/192 [==============================] - 14s 72ms/step - loss: 0.0922 - accuracy: 0.9659 - val_loss: 0.2380 - val_accuracy: 0.9191
Epoch 4/10
192/192 [==============================] - 13s 70ms/step - loss: 0.0639 - accuracy: 0.9792 - val_loss: 0.2682 - val_accuracy: 0.9107
Epoch 5/10
192/192 [==============================] - 16s 81ms/step - loss: 0.0485 - accuracy: 0.9829 - val_loss: 0.3190 - val_accuracy: 0.9153
Epoch 6/10
192/192 [==============================] - 17s 90ms/step - loss: 0.0286 - accuracy: 0.9915 - val_loss: 0.3708 - val_accuracy: 0.9074
Epoch 7/10
192/192 [==============================] - 14s 73ms/step - loss: 0.0171 - accuracy: 0.9955 - val_loss: 0.4264 - val_accuracy:

### Performance Metrics And Accuracy

In [35]:

y_pred1=model1.predict(X_test)

189/189 [==============================] - 4s 9ms/step


In [42]:
y_pred_class=[1 if y_pred1[i]>0.5 else 0 for i in range(len(y_pred1))]

In [40]:
from sklearn.metrics import confusion_matrix

In [43]:
confusion_matrix(y_test,y_pred_class)

array([[3146,  273],
       [ 329, 2287]])

In [46]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_class)

0.9002485501242751

In [49]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred_class))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3419
           1       0.89      0.87      0.88      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

